In [1]:
from data import *
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import os
import os.path as osp
import sys
import time

import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.nn import functional as F
import torchvision as tv
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets

import numpy as np
from PIL import Image
import argparse
from matplotlib import pyplot as plt
import pandas as pd
import cv2 
import pickle as pkl
import random
import tarfile
import collections
import math
import datetime

In [2]:
if sys.version_info[0] == 2:
    import xml.etree.cElementTree as ET
else:
    import xml.etree.ElementTree as ET

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
def adjust_learning_rate(optimizer, gamma, step):
    """Sets the learning rate to the initial LR decayed by 10 at every
        specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    lr = lr * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def xavier(param):
    init.xavier_uniform_(param)


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        m.bias.data.zero_()


def create_vis_plot(_xlabel, _ylabel, _title, _legend):
    return viz.line(
        X=torch.zeros((1,)).cpu(),
        Y=torch.zeros((1, 3)).cpu(),
        opts=dict(
            xlabel=_xlabel,
            ylabel=_ylabel,
            title=_title,
            legend=_legend
        )
    )


def update_vis_plot(iteration, loc, conf, window1, window2, update_type,
                    epoch_size=1):
    viz.line(
        X=torch.ones((1, 3)).cpu() * iteration,
        Y=torch.Tensor([loc, conf, loc + conf]).unsqueeze(0).cpu() / epoch_size,
        win=window1,
        update=update_type
    )
    # initialize epoch plot on first iteration
    if iteration == 0:
        viz.line(
            X=torch.zeros((1, 3)).cpu(),
            Y=torch.Tensor([loc, conf, loc + conf]).unsqueeze(0).cpu(),
            win=window2,
            update=True
        )

In [12]:
def train():
    
    cfg = voc
    dataset = VOCDetection(root=dataset_root, image_sets=[('2012', 'train')],
                            transform=SSDAugmentation(cfg['min_dim'],
                             MEANS))
    
#     visdom = False
#     if visdom:
#         import visdom
#         viz = visdom.Visdom()

    ssd_net = build_ssd('train', cfg['min_dim'], cfg['num_classes'])
    net = ssd_net

    if cuda:
        net = torch.nn.DataParallel(ssd_net)
        cudnn.benchmark = True

    if resume:
        print('Resuming training, loading {}...'.format(resume))
        ssd_net.load_weights(resume)
    else:
        vgg_weights = torch.load(basenet)
        print('Loading base network...')
        ssd_net.vgg.load_state_dict(vgg_weights)

    if cuda:
         net = net.to(device)

    if not resume:
        print('Initializing weights...')
        # initialize newly added layers' weights with xavier method
        ssd_net.extras.apply(weights_init)
        ssd_net.loc.apply(weights_init)
        ssd_net.conf.apply(weights_init)

    optimizer = optim.SGD(net.parameters(), lr, momentum,weight_decay)
    criterion = MultiBoxLoss(cfg['num_classes'], 0.5, True, 0, True, 3, 0.5,
                             False, cuda)

    net.train()
    name = 'train'
    
    # loss counters
    loc_loss = 0
    conf_loss = 0
    epoch = 0
    print('Loading the dataset...')

    epoch_size = len(dataset) // batch_size
    print('Training SSD on: ',name)
#    print('Using the specified args:')
#     print(args)

    step_index = 0

#     if visdom:
#         vis_title = 'SSD.PyTorch on ' + dataset.name
#         vis_legend = ['Loc Loss', 'Conf Loss', 'Total Loss']
#         iter_plot = create_vis_plot('Iteration', 'Loss', vis_title, vis_legend)
#         epoch_plot = create_vis_plot('Epoch', 'Loss', vis_title, vis_legend)

    data_loader = data.DataLoader(dataset, batch_size=32, num_workers=4, shuffle=True,\
                               collate_fn=detection_collate,pin_memory=True)
    
    print("Number of images in the training set = " + str(len(dataset)))
    print("Number of images in a mini-batch = "+str(batch_size))
    print("Number of mini-batches = " + str(len(data_loader)))
    
    
     # create batch iterator
    batch_iterator = iter(data_loader)
    print("STARTING - ITERATIONS")
    
    # Stats for pickle
    l_loss = []
    c_loss = []
    itr = []
    
    for iteration in range(start_iter, 10000):
        
        if visdom and iteration != 0 and (iteration % epoch_size == 0):
            update_vis_plot(epoch, loc_loss, conf_loss, epoch_plot, None,
                             'append', epoch_size)
            # reset epoch loss counters
            loc_loss = 0
            conf_loss = 0
            epoch += 1

        if iteration in cfg['lr_steps']:
            step_index += 1
            adjust_learning_rate(optimizer, gamma, step_index)

            
        ## load train data
        #images, targets = next(batch_iterator)
        try:
            images, targets = next(batch_iterator)
        except StopIteration:
            batch_iterator = iter(data_loader)
            images, targets = next(batch_iterator)


        
        if cuda:
            images = Variable(images.cuda())
            targets = [Variable(ann.cuda(), volatile=True) for ann in targets]
        else:
            images = Variable(images)
            targets = [Variable(ann, volatile=True) for ann in targets]
        
        # forward
        t0 = time.time()
        out = net(images)

        # backprop
        optimizer.zero_grad()

        loss_l, loss_c = criterion(out, targets)
        loss = loss_l + loss_c

        loss.backward()

        optimizer.step()

        t1 = time.time()
        loc_loss += loss_l.data.item()
        conf_loss += loss_c.data.item()

        l_loss.append(loss_l.data.item())
        c_loss.append(loss_c.data.item())
        itr.append(iteration)
        
        if iteration % 10 == 0:
            print('timer: %.4f sec.' % (t1 - t0))
            print('iter ' + repr(iteration) + ' || Loss: %.4f ||' % (loss.data.item()), end=' ')
            currentDT = datetime.datetime.now()
            print (currentDT.strftime("%H:%M:%S %p"))
            print("\n")
            
#         if visdom:
#             update_vis_plot(iteration, loss_l.data.item(), loss_c.data.item(),
#                             iter_plot, epoch_plot, 'append')

        
        if iteration != 0 and iteration % 10 == 0:
            print('Saving state, iter:', iteration)
            iter_name = math.ceil(iteration/100)*100
            torch.save(ssd_net.state_dict(), 'weights/ssd300_COCO_' +repr(iter_name) + '.pth')
            with open('stats_SGD.pkl','wb') as f:
                pkl.dump([l_loss, c_loss, itr], f)
                

    torch.save(ssd_net.state_dict(),
               save_folder + data_set + '.pth')

In [13]:
data_set = 'VOC'
dataset_root = '//datasets/ee285f-public/PascalVOC2012/'
basenet = 'weights/vgg16_reducedfc.pth'
batch_size = 32
resume = None
start_iter = 0
num_workers = 4
cuda = True
learning_rate = lr = 1e-3
momentum = 0.9
weight_decay = 5e-4
gamma = 0.1
visdom = False
save_folder = 'trained_weights/'
if not os.path.exists(save_folder):
    os.mkdir(save_folder)

# resume = 'ssd300_mAP_77.43_v2.pth'

In [ ]:
train()

Loading base network...
Initializing weights...
Loading the dataset...
Training SSD on:  train
Number of images in the training set = 5717
Number of images in a mini-batch = 32
Number of mini-batches = 179
STARTING - ITERATIONS


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:107: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/opt/conda/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


timer: 5.1993 sec.
iter 0 || Loss: 26.4353 || 20:02:55 PM


timer: 1.4019 sec.
iter 10 || Loss: 16.4076 || 20:05:11 PM


Saving state, iter: 10
timer: 1.9969 sec.
iter 20 || Loss: 15.7623 || 20:07:51 PM


Saving state, iter: 20
timer: 2.0943 sec.
iter 30 || Loss: 13.5455 || 20:10:21 PM


Saving state, iter: 30
timer: 1.9003 sec.
iter 40 || Loss: 14.4571 || 20:13:05 PM


Saving state, iter: 40
timer: 1.7958 sec.
iter 50 || Loss: 11.5304 || 20:15:28 PM


Saving state, iter: 50
timer: 1.5102 sec.
iter 60 || Loss: 10.6264 || 20:18:31 PM


Saving state, iter: 60
timer: 1.6024 sec.
iter 70 || Loss: 10.9013 || 20:20:48 PM


Saving state, iter: 70


In [14]:
# with open('stats.pkl','rb') as f:
#     l_loss, c_loss, itr = pkl.load(f)